In [ ]:
import requests
import json
import math
import pandas as pd
import hashlib
import sqlite3

In [ ]:
token='4H......sR.........iVz.......Mz'
header={'Authorization': f'Bearer {token}'}

In [ ]:
formList='https://api.typeform.com/forms?page_size=200&page={page}'
formItems='https://api.typeform.com/forms/{id}'
respList='https://api.typeform.com/forms/{id}/responses?page_size={psize}&page={page}'

In [ ]:
response=requests.get(formList.format(page=1),
                      headers=header).json()

forms=[]

for f in response['items']:
    form={}
    form['id']=f['id']
    form['url']=f['_links']['display']
    form['title']=f['title']
    
    forms.append(form)

f=pd.DataFrame(columns=forms[0].keys())
forms=f.append(forms)
del f
forms.set_index('id',inplace=True)
forms.head()

In [ ]:
forms.index

In [ ]:
forms=forms.loc[['fHUNzp']] #, 'pwnDwc', 'QAnf1l', 'KPbhd6']]

In [ ]:
field  = {}
fields = []

formItemsColumns=['id','form','type','title']

for form in forms.index:
    r=requests.get(formItems.format(id=form),
           headers=header).json()
    if 'fields' in r:
        for f in r['fields']:
            field = {}
            field['form'] = form
            field['id'] = f['id']
            field['title'] = f['title']
            field['type'] = f['type']
            if 'description' in f:
                field['description'] = f['description']

            fields.append(field)

    if 'hidden' in r:
        for f in r['hidden']:
            idCalc=hashlib.new('shake_256')

            field = {}
            field['form'] = form
            field['title'] = f
            field['type'] = 'hidden'
            
            idCalc.update('{}hidden{}'.format(form,f).encode('utf-8'))
            field['id'] = idCalc.hexdigest(5)

            fields.append(field)

            

formItems=pd.DataFrame(columns=formItemsColumns)
formItems=formItems.append(fields)
formItems.set_index('id',inplace=True)

In [ ]:
formItems

In [ ]:
meta     = {}
metas    = []
answer   = {}
answers  = []

metaColumns=['id', 'form_id', 'landed', 'submitted', 'agent', 'referer']
answerColumns=['id', 'form_id', 'response_id', 'field_id', 'answer']


for form in forms.index:
    responses=requests.get(respList.format(id=form,psize=1,page=1),
               headers=header).json()
    for page in range(1,math.ceil(responses['total_items']/1000)):
        responseSet=requests.get(respList.format(id=form,psize=1000,page=page),
                   headers=header).json()

        
        for i in responseSet['items']:
#             print(i)
            meta = {}
            meta['id']          = i['response_id']
            meta['form_id']     = form
            meta['landed']      = i['landed_at']
            meta['submitted']   = i['submitted_at']
            meta['agent']       = i['metadata']['user_agent']
            meta['referer']     = i['metadata']['referer']

            metas.append(meta)

            if 'hidden' in i.keys():
                for field in i['hidden'].keys():
                    idCalc=hashlib.new('shake_256')
                    answer = {}
                    answer['response_id'] =  meta['id']
                    answer['form_id']     =  form
                    idCalc.update('{}{}{}'.format(answer["form_id"],answer["response_id"],field).encode('utf-8'))
                    answer['id']          =  idCalc.hexdigest(5)
                    answer['answer']      =  i['hidden'][field]

                    
                    
                    idCalc=hashlib.new('shake_256')
                    idCalc.update('{}hidden{}'.format(answer["form_id"],field).encode('utf-8'))
                    answer['field_id']    =  idCalc.hexdigest(5)
                    
                    answers.append(answer)
                    

                
            if 'answers' in i.keys():
                for field in i['answers']:
#                     print(f'\t{field}')
                    idCalc=hashlib.new('shake_256')
    
                    answer = {}
                    answer['response_id'] =  meta['id']
                    answer['form_id']     =  form
                    answer['field_id']    =  field['field']['id']
                    idCalc.update('{}{}{}'.format(answer["form_id"],answer["response_id"],answer["field_id"]).encode('utf-8'))
                    answer['id']          =  idCalc.hexdigest(5)


                    if field['type'] == 'choices':
                        answer['answer']=[]
                        for k in field[field['type']].keys():
                            for a in field[field['type']][k]:
                                answer['answer'].append(a)
                        answer['answer']='|'.join(answer['answer'])
                    elif field['type'] == 'choice':
                        for k in field[field['type']].keys():
                            answer['answer']=field[field['type']][k]
                    else:
                        answer['answer'] = field[field['type']]

                    answers.append(answer)


meta=pd.DataFrame(columns=metaColumns)
if len(metas)>0:
    meta=meta.append(metas)
meta.set_index('id',inplace=True)
del metas

answer=pd.DataFrame(columns=answerColumns)
if len(answers)>0:
    answer=answer.append(answers)
answer.set_index('id',inplace=True)
del answers

In [ ]:
meta.head()

In [ ]:
answer.head(100)

## Grava o DataFrame para um DB SQLite
Este é o código para abrir ou criar um aquivo SQLite chamado `VivaReal.db`, na tabela `venda`. Para usar outro banco, como MariaDB, Oracle, DB2, altere esta célula e use a biblioteca SQL Alchemy com os drivers corretos. Todo o resto do código funcionará igual.

In [ ]:
dbfile = 'TypeForm.db'

db = sqlite3.connect(dbfile)

In [ ]:
forms.to_sql('forms', index=True, if_exists='replace', con=db)
formItems.to_sql('formitems', index=True, if_exists='replace', con=db)
meta.to_sql('responses', index=True, if_exists='replace', con=db)
answer.to_sql('responseitems', index=True, if_exists='replace', con=db)

In [ ]:
db.commit()
db.close()